# Chapter 3 Evaluation Input - Classification

- [I. Environment Configuration](#I. Environment Configuration)
- [II. Classify User Instructions](#II. Classify User Instructions)

In this chapter, we will focus on evaluating input tasks, which is critical to ensuring the quality and security of the system.

For tasks that require handling many independent sets of instructions in different situations, it is beneficial to first classify the query type and use this as a basis to determine which instructions to use.

This can be achieved by defining fixed categories and hard-coding the instructions that are relevant to handling tasks in a given category.

For example, when building a customer service assistant, it may be important to first classify the query type and then determine which instructions to use based on that classification.

As a specific example, if the user asks to close their account, then the secondary instruction may be to add additional instructions on how to close the account; whereas if the user asks for information about a specific product, then the secondary instruction may be to add more product information.

## 1. Environment Configuration
As in the previous chapter, we first need to configure the environment to use the OpenAI API

In [9]:
import openai
# Import third-party libraries

openai.api_key  = "sk-..."
# Set API_KEY, please replace it with your own API_KEY

# The following is an example of a configuration method based on environment variables, which is safer. It is for reference only and will not be covered later.
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [2]:
def get_completion_from_messages(messages, 
                                model="gpt-3.5-turbo", 
                                temperature=0, 
                                max_tokens=500):
    '''
    封装一个访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

## 2. Classify user instructions

Here, we use system_message as the global guide for the system and choose `#` as the delimiter.

A delimiter is a tool used to distinguish different parts of a command or output. It helps the model identify the parts, thereby improving the accuracy and efficiency of the system in performing specific tasks.

In this example, we choose to use `#` as the delimiter.

`#` is an ideal delimiter because it can be treated as an independent token.

In [3]:
delimiter = "####"

This is our system message and we are asking the model in the following way.

In [4]:
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""

In [5]:
system_message = f"""
你将获得客户服务查询。
每个客户服务查询都将用{delimiter}字符分隔。
将每个查询分类到一个主要类别和一个次要类别中。
以 JSON 格式提供你的输出，包含以下键：primary 和 secondary。

主要类别：计费（Billing）、技术支持（Technical Support）、账户管理（Account Management）或一般咨询（General Inquiry）。

计费次要类别：
取消订阅或升级（Unsubscribe or upgrade）
添加付款方式（Add a payment method）
收费解释（Explanation for charge）
争议费用（Dispute a charge）

技术支持次要类别：
常规故障排除（General troubleshooting）
设备兼容性（Device compatibility）
软件更新（Software updates）

账户管理次要类别：
重置密码（Password reset）
更新个人信息（Update personal information）
关闭账户（Close account）
账户安全（Account security）

一般咨询次要类别：
产品信息（Product information）
定价（Pricing）
反馈（Feedback）
与人工对话（Speak to a human）

"""

Now let's look at an example of a user message.

In [26]:
user_message = f"""\ 
I want you to delete my profile and all of my user data"""

In [6]:
user_message = f"""\ 
我希望你删除我的个人资料和所有用户数据。"""

Format this message as a list of messages, with system messages and user messages separated by "####".

Let's think about which category this sentence belongs to as a human: "I want you to delete my profile."

This sentence looks like it belongs to "Account Management" or "Close account".

In [7]:
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]

Let's see how the model thinks

The model's classification is "Account Management" as "primary" and "Close account" as "secondary".

The benefit of requesting a structured output like JSON is that you can easily read it into an object, such as a dictionary in Python. If you use another language, you can also convert to another object and then input it into the subsequent steps.

In [10]:
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "账户管理",
    "secondary": "关闭账户"
}


Here is another user message: "Tell me more about your flat screen TV"

We apply the same list of messages to get the model's response and print it out.

Another classification result is returned here and it looks like it should be correct.

In [31]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}


In [12]:
user_message = f"""\
告诉我更多有关你们的平板电脑的信息"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

以下是针对平板电脑的一般咨询：

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}

如果您有任何特定的问题或需要更详细的信息，请告诉我，我会尽力回答。


So, depending on the classification of the customer inquiry, we can now provide a more specific set of instructions to handle the next steps.

In this case, we might add additional information about the TV, while in other cases, we might want to provide a link to close the account or something similar.

In the following chapters, we will take a closer look at different ways to handle input

In the next chapter, we will explore more about ways to evaluate input, specifically how to ensure that users use the system in a responsible manner.